In [ ]:
import sys
from pathlib import Path
from os import getcwd, remove, listdir, stat, chdir
from os.path import dirname, join, isfile, isdir, basename
sys.path.append(str(Path(getcwd()).parent.parent))
import pyminizip as pz
from datetime import date, datetime, timedelta

import pandas as pd
import numpy as np
from glob import glob
from time import sleep
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')


from bulletin import setup_log, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin import root, default_input, default_output, tables_path
from bulletin.systems.esus import eSUS
from bulletin.utils.utils import strlist
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.timer import Timer
from bulletin.utils import setup_logging

from bulletin.services.connections import get_db_conn

logger = setup_logging(setup_log,name='vacinados_1_generate_eSUSdb')

tables = dict([(Path(x).stem,pd.read_csv(join(tables_path,x))) for x in glob(join(tables_path,"*.csv"))])

In [ ]:
situacao_labels = { 0:'Não Vacinado', 1:'Esquema Primário Incompleto', 2:'Esquema Primário Completo ', 3: 'Esquema Primário Completo + Reforço'}
mes_labels = {key+1:value for key,value in enumerate(['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'])}

In [ ]:
download_all = True
read_raw = True

update = True

esus = eSUS()

if download_all and not read_raw:
    esus.download_all()

elif read_raw:
        esus.load('doses_aplicadas_raw',compress=True)
        esus.normalize()
        esus.save('doses_aplicadas',replace=True)
else:    
    esus.load()

if update:
    update_part = esus.download_update(interval=14,load_downloaded=False)
    esus.update(update_part)
    esus.save(replace=True)

esus.df

In [ ]:
# Testar banco
# interval = 7
# with get_db_conn(esus.config) as conn:
#     query = f'''
#         SELECT * FROM imunizacao_covid
#         WHERE ("vacina_dataAplicacao" >= NOW() - INTERVAL '{interval} DAY') 
#         OR ("data_importacao_rnds" >= NOW() - INTERVAL '{interval} DAY') 
#         OR ("dt_deleted" >= NOW() - INTERVAL '{interval} DAY') 
#     '''
#     logger.info(query)
#     df = pd.read_sql(query, conn)
# df

In [ ]:
esus.df.groupby(['dose','vacina_descricao_dose'],dropna=False)[['paciente']].count()

In [ ]:
error = esus.df.loc[esus.df['status']=='entered-in-error']
esus.df = esus.df.drop(index=error.index)
error.shape

In [ ]:
# esus.df.rename(columns={
#             'estabelecimento_municipio_codigo':'ibge_atendimento',
#             'vacina_dataAplicacao': 'data_aplicacao',
#             'vacina_fabricante_nome': 'fabricante',
#             'vacina_categoria_nome': 'categoria',
#             'vacina_grupoAtendimento_nome': 'grupo_atendimento'
#         }, inplace=True)

In [ ]:
duplicate_cols = esus.df.columns[esus.df.columns.duplicated()]
# esus.df.drop(columns=duplicate_cols, inplace=True)
esus.df[duplicate_cols]

In [ ]:
data_aplicacao_dose = esus.df.groupby(['data_aplicacao','dose'])[['paciente']].count().unstack('dose').fillna(0).astype(int)
data_aplicacao_dose.rolling(14).mean().plot(figsize=(15,10))

In [ ]:
esus.df.groupby(['ibge_atendimento','vacina_profissionalAplicador_nome'],dropna=True)[['document_id']].count().sort_values('document_id',ascending=False).head(10)

In [ ]:
verification = esus.df['cns'].isna()
print(f"{verification.sum()} registros com cns nulo")
if verification.sum() > 0: esus.df.drop(index=esus.df.loc[verification].index, inplace=True)
del verification

In [ ]:
esus.df.drop(columns=['document_id','raca_cor', 'email_paciente',
       'telefone_paciente', 'vacina_descricao_dose', 'data_importacao_rnds',
       'estabelecimento_municipio_nome',
       'paciente_endereco_coPais', 'estabelecimento_uf',
       'bairro_residencia', 'paciente_endereco_nmPais',
       'paciente_endereco_nmMunicipio', 'vacina_lote', 'uf_residencia',
       'estalecimento_noFantasia','id_sistema_origem',
       'vacina_profissionalAplicador',
       'vacina_profissionalAplicador_nome', 'vacina_codigo',
       'estabelecimento_valor','paciente_nacionalidade_enumNacionalidade',
       'cep_residencia', 'timestamp', 'paciente_racaCor_codigo',
       'estabelecimento_razaoSocial', 'vacina_grupoAtendimento_codigo',
       'vacina_categoria_codigo', 'dt_deleted','status','vacina_fabricante_referencia']
,inplace=True)
esus.df.columns

In [ ]:
dose1 = esus.df.loc[(esus.df['dose']=='1ª Dose')]
dose2 = esus.df.loc[esus.df['dose']=='2ª Dose']
doseUnica = esus.df.loc[esus.df['dose']=='Única']
doseAdicional = esus.df.loc[(esus.df['dose']=='Adicional')]
doseReforco = esus.df.loc[(esus.df['dose']=='Reforço')]

In [ ]:
del esus.df
dose1

In [ ]:
cols = ['paciente_id','paciente','cpf','sexo','nome_mae','data_nascimento','idade','ibge_residencia','categoria','grupo_atendimento','ibge_atendimento','estabelecimento_razaoSocial']

vacinados = pd.merge(dose1,dose2,on='cns',how='outer',suffixes=['_1a_dose','_2a_dose'])

for col in cols:
    vacinados.loc[vacinados[f"{col}_1a_dose"].notna(), col] = vacinados.loc[vacinados[f"{col}_1a_dose"].notna(), f"{col}_1a_dose"]
    vacinados.loc[(vacinados[f"{col}_1a_dose"].isna()) & (vacinados[f"{col}_2a_dose"].notna()), col] = vacinados.loc[(vacinados[f"{col}_1a_dose"].isna()) & (vacinados[f"{col}_2a_dose"].notna()), f"{col}_2a_dose"]

del dose1
del dose2
# vacinados['paciente_id']

In [ ]:
doseUnica = doseUnica.rename(columns={'vacina_nome':'vacina_nome_dose_unica', 'data_aplicacao':'data_aplicacao_dose_unica', 'dose':'dose_unica'})

vacinados = pd.merge(vacinados,doseUnica,on='cns',how='outer',suffixes=['','_dose_unica'])

for col in cols:
    vacinados.loc[(vacinados[col].isna()) & (vacinados[f"{col}_dose_unica"].notna()), col] = vacinados.loc[(vacinados[col].isna()) & (vacinados[f"{col}_dose_unica"].notna()), f"{col}_dose_unica"]

del doseUnica
# vacinados['paciente_id']

In [ ]:
doseAdicional = doseAdicional.rename(columns={'vacina_nome':'vacina_nome_dose_adicional', 'data_aplicacao':'data_aplicacao_dose_adicional', 'dose':'dose_adicional'})

vacinados = pd.merge(vacinados,doseAdicional,on='cns',how='outer',suffixes=['','_dose_adicional'])

for col in cols:
    vacinados.loc[(vacinados[col].isna()) & (vacinados[f"{col}_dose_adicional"].notna()), col] = vacinados.loc[(vacinados[col].isna()) & (vacinados[f"{col}_dose_adicional"].notna()), f"{col}_dose_adicional"]

del doseAdicional
# vacinados['paciente_id']

In [ ]:
doseReforco = doseReforco.rename(columns={'vacina_nome':'vacina_nome_dose_reforco', 'data_aplicacao':'data_aplicacao_dose_reforco', 'dose':'dose_reforco'})

vacinados = pd.merge(vacinados,doseReforco,on='cns',how='outer',suffixes=['','_dose_reforco'])

for col in cols:
    vacinados.loc[(vacinados[col].isna()) & (vacinados[f"{col}_dose_reforco"].notna()), col] = vacinados.loc[(vacinados[col].isna()) & (vacinados[f"{col}_dose_reforco"].notna()), f"{col}_dose_reforco"]

del doseReforco
# vacinados['paciente_id']

In [ ]:
vacinados['doses_aplicadas'] = vacinados['dose_1a_dose'].fillna('NA') +','+ vacinados['dose_2a_dose'].fillna('NA') +','+ vacinados['dose_unica'].fillna('NA') +','+ vacinados['dose_adicional'].fillna('NA') +','+ vacinados['dose_reforco'].fillna('NA')
vacinados['doses_aplicadas'] = vacinados['doses_aplicadas'].str.replace('NA,','').str.replace(',NA','')
vacinados.groupby(['doses_aplicadas'])[['cns']].count().sort_values('cns', ascending=False)

In [ ]:
used_cols = ['cns','paciente','cpf','sexo','nome_mae','data_nascimento','idade','ibge_residencia','ibge_atendimento','categoria','grupo_atendimento','doses_aplicadas','vacina_nome_1a_dose','data_aplicacao_1a_dose','vacina_nome_2a_dose','data_aplicacao_2a_dose','vacina_nome_dose_unica','data_aplicacao_dose_unica','vacina_nome_dose_adicional','data_aplicacao_dose_adicional','vacina_nome_dose_reforco','data_aplicacao_dose_reforco']
# used_cols = ['pacient_id','cns','paciente','cpf','sexo','nome_mae','data_nascimento','idade','ibge_residencia','ibge_atendimento','categoria','grupo_atendimento','doses_aplicadas','vacina_nome_1a_dose','data_aplicacao_1a_dose','vacina_nome_2a_dose','data_aplicacao_2a_dose','vacina_nome_dose_unica','data_aplicacao_dose_unica','vacina_nome_dose_adicional','data_aplicacao_dose_adicional','vacina_nome_dose_reforco','data_aplicacao_dose_reforco']
vacinados = vacinados.drop(columns=[col for col in vacinados.columns if not col in used_cols])
vacinados = vacinados.drop_duplicates('cns', keep='first')
vacinados = vacinados[used_cols]

In [ ]:
# inconsistencia
vacinados['situacao_atual'] = 0

# esquema vacinal incompleto
vacinados.loc[
    (vacinados['doses_aplicadas']=='1ª Dose')
,'situacao_atual'] =  1

# esquema vacinal completo 
vacinados.loc[
    (vacinados['doses_aplicadas']=='1ª Dose,2ª Dose') | ## corretos
    (vacinados['doses_aplicadas']=='1ª Dose,2ª Dose,Adicional') |
    (vacinados['doses_aplicadas']=='Única') | 
    (vacinados['doses_aplicadas']=='Única,Adicional') |

    (vacinados['doses_aplicadas']=='1ª Dose,2ª Dose,Única') | # "inconsistentes"
    (vacinados['doses_aplicadas']=='2ª Dose,Única') |            
    (vacinados['doses_aplicadas']=='2ª Dose') |              
    (vacinados['doses_aplicadas']=='1ª Dose,Única')|
    (vacinados['doses_aplicadas']=='1ª Dose,2ª Dose,Única,Adicional') | 
    (vacinados['doses_aplicadas']=='1ª Dose,Adicional') |
    (vacinados['doses_aplicadas']=='1ª Dose,Única,Adicional') |
    (vacinados['doses_aplicadas']=='2ª Dose,Adicional') |
    (vacinados['doses_aplicadas']=='2ª Dose,Única,Adicional') |
    (vacinados['doses_aplicadas']=='Adicional') 
,'situacao_atual'] = 2

# esquema vacinal completo + reforço
vacinados.loc[
    
    (vacinados['doses_aplicadas']=='1ª Dose,2ª Dose,Reforço') | ## corretos
    (vacinados['doses_aplicadas']=='1ª Dose,2ª Dose,Adicional,Reforço') |
    (vacinados['doses_aplicadas']=='Única,Reforço') |
    (vacinados['doses_aplicadas']=='Única,Adicional,Reforço') |

    (vacinados['doses_aplicadas']=='1ª Dose,2ª Dose,Única,Reforço') | ## "inconsistentes"
    (vacinados['doses_aplicadas']=='1ª Dose,Reforço') |
    (vacinados['doses_aplicadas']=='1ª Dose,Única,Reforço') |
    (vacinados['doses_aplicadas']=='2ª Dose,Reforço') |
    (vacinados['doses_aplicadas']=='2ª Dose,Única,Reforço') |
    (vacinados['doses_aplicadas']=='Reforço') |
    (vacinados['doses_aplicadas']=='1ª Dose,Adicional,Reforço') |
    (vacinados['doses_aplicadas']=='2ª Dose,Adicional,Reforço') |
    (vacinados['doses_aplicadas']=='1ª Dose,Única,Adicional,Reforço') |
    (vacinados['doses_aplicadas']=='2ª Dose,Única,Adicional,Reforço') |
    (vacinados['doses_aplicadas']=='1ª Dose,2ª Dose,Única,Adicional,Reforço') |
    (vacinados['doses_aplicadas']=='Adicional,Reforço')     
,'situacao_atual'] = 3     

In [ ]:
situacao_atual = vacinados.groupby(['situacao_atual','doses_aplicadas'])[['cns']].count().reset_index().rename(columns={'cns':'pacientes'})#.sort_values('cns', ascending=False)
situacao_atual['n_doses_aplicadas'] = situacao_atual['doses_aplicadas'].str.split(',').apply(len)
situacao_atual['aplicacoes'] = (situacao_atual['n_doses_aplicadas'] * situacao_atual['pacientes'])
del situacao_atual['n_doses_aplicadas']
situacao_atual = situacao_atual.set_index(['situacao_atual','doses_aplicadas'])
situacao_atual.loc[('','TOTAL'),:] = situacao_atual.sum(axis=0)
display(situacao_atual.groupby('situacao_atual').sum())
situacao_atual.astype(float).rename(index=situacao_labels, level='situacao_atual')

In [ ]:
vacinados['data_ultima_dose'] = vacinados[[data for data in vacinados.columns if 'data_aplicacao' in data]].max(axis=1)
vacinados['dias_apos_ultima_dose'] = (vacinados['data_ultima_dose'] - hoje).dt.days * -1

In [ ]:
tempo_imunizacao = 15
tempo_imunizacao_reforco = 15

vacinados.loc[vacinados['data_aplicacao_2a_dose'].notna(),'data_imunizacao'] = vacinados.loc[vacinados['data_aplicacao_2a_dose'].notna(),'data_aplicacao_2a_dose'] + timedelta(tempo_imunizacao)
vacinados.loc[vacinados['data_aplicacao_dose_unica'].notna(),'data_imunizacao'] = vacinados.loc[vacinados['data_aplicacao_dose_unica'].notna(),'data_aplicacao_dose_unica'] + timedelta(tempo_imunizacao)
vacinados.loc[vacinados['data_aplicacao_dose_adicional'].notna(),'data_imunizacao'] = vacinados.loc[vacinados['data_aplicacao_dose_adicional'].notna(),'data_aplicacao_dose_adicional'] + timedelta(tempo_imunizacao_reforco)
vacinados.loc[vacinados['data_aplicacao_dose_reforco'].notna(),'data_imunizacao_reforco'] = vacinados.loc[vacinados['data_aplicacao_dose_reforco'].notna(),'data_aplicacao_dose_reforco'] + timedelta(tempo_imunizacao_reforco)

In [ ]:
vacinados = vacinados.join(tables['municipios'][['ibge','uf']].set_index('ibge'), on='ibge_residencia')

In [ ]:
# vacinados.loc[(vacinados['data_nascimento'].notnull()) & (vacinados['data_ultima_dose'].notnull()), 'idade'] = \
#     vacinados.loc[(vacinados['data_nascimento'].notnull()) & (vacinados['data_ultima_dose'].notnull())].apply(
#             lambda row: row['data_ultima_dose'].year - row['data_nascimento'].year - (
#                     (row['data_ultima_dose'].month, row['data_ultima_dose'].day) <
#                     (row['data_nascimento'].month, row['data_nascimento'].day)
#             ), axis=1
#     )

In [ ]:
vacinados['fx'] = '60+'
vacinados.loc[(vacinados['idade']>=12)&(vacinados['idade']<60),'fx'] = '12-59'
vacinados.loc[(vacinados['idade']<12),'fx'] = '0-11'

In [ ]:
vac = eSUS('vacinados')
vac.df = vacinados
vac.hashes()
vac.df.index = np.arange(len(vac.df))
vac.save(replace=True, compress=True)

In [ ]:
# vac = eSUS('vacinados')
# vac.load()
# vac.df.shape
# vacinados = vac.df

In [ ]:
situacao_atual = vacinados.groupby(['situacao_atual','doses_aplicadas'])[['cns']].count().reset_index().rename(columns={'cns':'pacientes'})#.sort_values('cns', ascending=False)
situacao_atual['n_doses_aplicadas'] = situacao_atual['doses_aplicadas'].str.split(',').apply(len)
situacao_atual['aplicacoes'] = (situacao_atual['n_doses_aplicadas'] * situacao_atual['pacientes'])
del situacao_atual['n_doses_aplicadas']
situacao_atual = situacao_atual.set_index(['situacao_atual','doses_aplicadas'])
situacao_atual.loc[('','TOTAL'),:] = situacao_atual.sum(axis=0)
display(situacao_atual.groupby('situacao_atual').sum().rename(index=situacao_labels, level='situacao_atual'))
situacao_atual.astype(float).rename(index=situacao_labels, level='situacao_atual').to_excel('situacao_atual.xlsx')
situacao_atual.astype(float).rename(index=situacao_labels, level='situacao_atual')

In [ ]:
display(vacinados.loc[vacinados['uf']=='PR'].shape)
display(vacinados.loc[vacinados['uf']!='PR'].shape)

In [ ]:
vacinados_pr = vacinados.loc[vacinados['uf']=='PR']
vacinados_mes_fx_situacao = vacinados_pr.groupby([vacinados_pr['data_ultima_dose'].rename('ano').dt.year,vacinados_pr['data_ultima_dose'].rename('mes').dt.month,'fx','situacao_atual'])[['cns']].count().unstack(['fx','situacao_atual']).fillna(0).droplevel(0,1).astype(int)
vacinados_mes_fx_situacao = vacinados_mes_fx_situacao.sort_index(axis=1,level='fx')
vacinados_mes_fx_situacao

In [ ]:
vacinados_mes_fx_situacao_acumulado = vacinados_mes_fx_situacao.cumsum()
vacinados_mes_fx_situacao_acumulado = vacinados_mes_fx_situacao_acumulado.sort_index(axis=1,level='fx')
vacinados_mes_fx_situacao_acumulado

In [ ]:
estimativa_populacional = [1868761, 7879614, 1927286] # [menor que 12, de 12 até 59, maior ou igual a 60]

populacao_mes = pd.DataFrame([estimativa_populacional], index=vacinados_mes_fx_situacao.index, columns=pd.MultiIndex.from_product([['0-11','12-59','60+'],[0]],names=['fx','situacao_atual']))
populacao_mes_nao_vacinada = populacao_mes - vacinados_mes_fx_situacao_acumulado.groupby(axis=1,level='fx').sum()
populacao_mes_nao_vacinada

In [ ]:
vacinados_e_nao_mes_fx_situacao_acumulado = pd.concat([vacinados_mes_fx_situacao_acumulado,populacao_mes_nao_vacinada], axis=1)
vacinados_e_nao_mes_fx_situacao_acumulado = vacinados_e_nao_mes_fx_situacao_acumulado.sort_index(axis=1,level='fx')
vacinados_e_nao_mes_fx_situacao_acumulado = vacinados_e_nao_mes_fx_situacao_acumulado.rename(columns=situacao_labels, level='situacao_atual').rename(index=mes_labels, level='mes')#.to_excel('vacinados_e_nao_mes_fx_situacao_acumulado.xlsx')
vacinados_e_nao_mes_fx_situacao_acumulado

In [ ]:
vacinados_e_nao_mes_fx_situacao_acumulado.iloc[-1].to_csv(join(default_output,'vacinacao','populacao_vacinada.csv'))

In [ ]:
vacinados[['cns','dias_apos_ultima_dose']].groupby('dias_apos_ultima_dose').count().plot()

In [ ]:
vacinados['faixa_etaria'] = np.digitize(vacinados['idade'],tables['faixa_etaria_populacao']['bins'],right=False)
vacinados[['cns','faixa_etaria']].groupby('faixa_etaria').count().plot(kind='bar')

In [ ]:
bins = [1,2,3,7,14,21,30,60,90]
bins_label = ['hoje','24 horas','48 horas', '72 horas', '7 dias', '14 dias', '21 dias', '30 dias', '60 dias', '90 dias']
vacinados['periodo_ultima_dose'] = [ x for x in np.digitize(vacinados['dias_apos_ultima_dose'],bins,right=False)]
dias_apos_ultima_dose = vacinados.groupby(['situacao_atual','periodo_ultima_dose'])[['cns']].count().unstack().fillna(0).astype(int).rename(columns={'cns':'qtde'}).droplevel(0,1)
dias_apos_ultima_dose['total'] = dias_apos_ultima_dose.sum(1)
dias_apos_ultima_dose = dias_apos_ultima_dose.append(pd.DataFrame(dias_apos_ultima_dose.sum().tolist(),index=dias_apos_ultima_dose.columns.tolist(),columns=['total']).T)
dias_apos_ultima_dose.columns = [ bins_label[bins] if isinstance(bins,int) else bins for bins in dias_apos_ultima_dose.columns ]
# dias_apos_ultima_dose = dias_apos_ultima_dose.reset_index()
dias_apos_ultima_dose

In [ ]:
del vacinados